In [1]:
#imports
import numpy as np
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt

from layered_neural_net import lnn



In [2]:
#Run_hyperparameters
layers = [28*28,200,10]


In [3]:
#load dataset
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  '  + str(test_X.shape))
print('Y_test:  '  + str(test_y.shape))

X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [4]:
#try on all classes
#one hot encode
def one_hot_encode(x):
    out = np.zeros((len(x), max(x)+1))
    for i in range(len(x)):
        out[i][x[i]] = 1
    return out

y_train_ohe = one_hot_encode(train_y)
y_test_ohe = one_hot_encode(test_y)
#flatten images
x_train = train_X.reshape(len(train_X), 28*28)
x_test = test_X.reshape(len(test_X), 28*28)
"""
x_test = np.array_split(x_test, 10)
y_test_ohe = np.array_split(y_test_ohe, 10)

"""


'\nx_test = np.array_split(x_test, 10)\ny_test_ohe = np.array_split(y_test_ohe, 10)\n\n'

In [5]:
x
print(x_test[2].shape)
print(y_test_ohe[2].shape)

print(x_train.shape)
print(y_train_ohe.shape)


NameError: name 'x' is not defined

In [ ]:
#setting up evolutionary pipeline
performance_over_time = []
#creating initial population
population_size = 10
def create_population(population_size):
    population = []
    print("Creating networks")
    for i in range(population_size):
        net = lnn(layers)
        population.append(net)
        print("|", end="")
    print("done!")
    return population


networks = create_population(population_size)

In [ ]:
def get_perf(t):
        return t[0]

#measure performance of all the networks
def evaluate_performance(population, x, y):
    performances = []
    print("evaluating performances", end="")
    for net in population:
        predictions = net.predict(x)[0]
        correct_count = len(x)-(np.sum(np.abs(y - predictions))/2)
        performances.append((correct_count/len(x), net))
        #print(correct_count/len(x))
        print("|", end="")
    print(" done!", end=" ")
    #sort by best performance
    performances.sort(key = get_perf, reverse = True)
    return performances

evaluated_networks = evaluate_performance(networks, x_test[1], y_test_ohe[1])
print("best performer of this generation :", evaluated_networks[0][0])
performance_over_time.append(np.array(evaluated_networks)[:,0])

In [ ]:
#the n best performing networks will be selected
n = 3
mutation_range = 0.1

def repopulate(evaluated_networks, mutation_range, n):
    offspring_per_network = int(population_size/n)
    parents = [i[1] for i in evaluated_networks[:n]]
    offspring = []
    for net in parents:
        net_offspring = net.reproduce(min(offspring_per_network, population_size - len(offspring)), mutation_range)
        offspring.extend(net_offspring)
    next_gen = parents + offspring
    next_gen = next_gen[:population_size]
    return next_gen
    
next_generation = repopulate(evaluated_networks, mutation_range,n)

In [ ]:
performance_over_time = []

In [ ]:
test_sets_used = []
mutation_ranges = []

In [ ]:
print(np.array(performance_over_time))
for i in performance_over_time:
    print(len(i))

In [ ]:
generations = 1
mutation_range = 0.05
population_size = 10
for gen in range(generations):
    #if(gen%10==0 and gen!=0): 
    #    mutation_range/=1.4
    #    print("decreasing mutation range from",mutation_range*1.4,"to",mutation_range)
    print("generation "+str(len(performance_over_time)+1), end=" ")
    test_set = np.random.randint(10)
    test_sets_used.append(test_set)
    print(" test set:",test_set,end=" ")
    evaluated_networks = evaluate_performance(next_generation, x_test[test_set], y_test_ohe[test_set])
    print(" best:", evaluated_networks[0][0], "second:", evaluated_networks[1][0], "third:", evaluated_networks[2][0])
    performance_over_time.append(list(np.array(evaluated_networks)[:,0]))
    generational_mutation_range = np.random.rand()*mutation_range
    mutation_ranges.append(generational_mutation_range)
    print("mutating in range:", generational_mutation_range)
    next_generation = repopulate(evaluated_networks, generational_mutation_range, n)
    if(gen%1000==0):
        plt.plot(performance_over_time, alpha= 0.1)
        plt.plot(np.array(performance_over_time)[:,0])
        plt.show()
        print("average best of last 1000 generations",np.average(np.array(performance_over_time)[-1000:,0]))
    

In [ ]:
#visualising performance across the 10 different sets

performance_hist = np.array(performance_over_time)[-len(test_sets_used):,0]
set_perf = []
for i in range(10):
    set_perf.append([])

for i in range(len(test_sets_used)):
    set_perf[test_sets_used[i]].append(performance_hist[i])

for i in set_perf:
    plt.plot(i, alpha=0.8)
plt.show()

plt.violinplot(set_perf)
plt.show()

performance_hist = np.array(performance_over_time)[-len(test_sets_used):]
performance_changes = []
for i in range(len(performance_hist)-1):
    performance_changes.append(np.average(performance_hist[i])-np.average(performance_hist[i+1]))

print(np.array(mutation_ranges)[:len(mutation_ranges)].shape)
print(np.array(performance_changes)[:len(mutation_ranges)].shape)

plt.scatter(np.array(performance_changes)[:len(mutation_ranges)], np.array(mutation_ranges)[:len(performance_changes)])
plt.show()
plt.violinplot(np.array(mutation_ranges))
plt.show()

In [ ]:
#TQDM -> loading bars

plt.plot(performance_over_time, alpha= 0.1)
plt.plot(np.array(performance_over_time)[:,0])
#plt.plot(list(0.99*np.ones(len(performance_over_time))))
plt.show()

In [ ]:
plt.plot(np.average(np.split(np.array(performance_over_time)[:6400,0], 100),axis =1))

import matplotlib.font_manager as fm

# Assuming 'performance_over_time' is already defined
average_performance = np.average(np.split(np.array(performance_over_time)[:6400,0], 100),axis=1)

# Create a larger figure
fig = plt.figure(figsize=(10, 6))

# Get the current axes, creating one if necessary.
ax = fig.gca()

# Set the background color of the plot to black
ax.set_facecolor('black')

# Create the plot with a line style and a bright blue color
plt.plot(average_performance, linestyle='-', color='cyan', linewidth=2.0)

# Specify a font
font_path = '/Users/hannes/Library/Fonts/CenturyGothic.ttf'
font = fm.FontProperties(fname=font_path, size=16)

# Add title and labels with larger font sizes and black color
plt.title('Average Performance Over Time', fontsize=20, color='black', fontproperties=font)
plt.xlabel('Time', fontsize=16, color='black', fontproperties=font)
plt.ylabel('Average Performance', fontsize=16, color='black', fontproperties=font)

# Add a grid for easier visualization of values, with blue color
plt.grid(True, color='dodgerblue')

# Increase the size of the ticks for the x and y axes and make them black
plt.xticks(fontsize=14, color='black', fontproperties=font)
plt.yticks(fontsize=14, color='black', fontproperties=font)

# Add a legend with black font
plt.legend(['Performance'], loc='upper left', fontsize=14, facecolor='black', edgecolor='dodgerblue', labelcolor='black')

# Set the color of the spines to blue
ax.spines[:].set_color('dodgerblue')

# Show the plot
plt.show()

In [ ]:
import matplotlib.font_manager
fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

for i in fonts:
    if "Gothic" in i:
        print(i)


In [ ]:
np.savetxt('training_run_'+str(training_run)+'_performance.csv', performance_over_time, delimiter=',')
np.savetxt('training_run_'+str(training_run)+'_test_sets_used.csv', test_sets_used, delimiter=',')
np.savetxt('training_run_'+str(training_run)+'_mutation_ranges.csv', mutation_ranges, delimiter=',')
np.save('training_run_'+str(training_run)+'_config.npy', np.array(log))

In [ ]:
#Ideas to improve GA
#take best of last 3 gens instead of best 3 of last gen
#Cap unused neurons (if collumn or line of adj matrix empty, kill both)
#topology modification